# Introduction
(150 - 200 words)

The long acquisition time in fully sampled MRI leads to low patient throughput, problems with patient comfort and compliance, artefacts from patient motion, and high exam costs. Reducing acquisition time by under-sampling helps to mitigate these issues, but at the cost of reconstructed image quality. The aim of this machine learning task is to improve the viability of the more efficient under-sampling strategy by designing a system that maximises the quality of under sampled reconstructions.

The MRI dataset provided contained raw k-space data from 100 3D volumes, each having approximately 30-40 2D slices. The data was split into training and testing sets with ratio 7:3 respectively. The training set contained only fully sampled k-space data from which ground truth and under-sampled data could be derived. The test set contained 4-fold and 8-fold under sampled k-space data as well as the 4-fold and 8-fold masks that generated this data.

# Design
(450 - 600 words)

After some through research of various types of neural networks, we decided that the best model for this project would be a U-Net. This is because, at the time of writing, U-Nets are regarded to be one of the best types of Convolutional Neural Networks for biomedical imaging.

The major attraction to U-Nets for such tasks is their apparent lack of overfitting risk.

[Ref 1] The performance of the U-Net models continues to increase with increasing model capacity, and 17 even the largest model with over 200 million parameters is unable to overfit the training data.

The other types of Neural Networks that we looked at for this task were: 
- Convolutional Neural Network
- Recurrent Neural Network
- 

We chose to experiment with the following hyperparameters (change as necessary):
- Epochs
- Learning Rate
- Dropout Probability
- Step Size
- Batch Size
- Pooling Layers

Guess we should explain what each are and why we c
In addition to this we altered the layers in the neural network...

# Implementation
(600 - 800 words)

Initially, we began implementing a U-Net from scratch however, quickly ran into problems that were quite ambiguous. After some research, we found a preexisting model provided in the facebook research fast MRI project (https://github.com/facebookresearch/fastMRI/tree/master/models/unet). 

By opting to use a preexisting model, we could ensure that our basic model configuration was correct. It also allowed us to deduce that any issues along the way were caused by parts of the code that were not directly related to the model class.

The initial implementation of the neural network had the following structure. The model carries out down-sampling and up-sampling, resulting in the formation of two deep convolutional networks within the U-Net.

### Optimizer

The optimizer we chose to use was ReLU (Rectified Linear Unit)...

### Test Validation Split

An imperative part of the task was to monitor for signs of overfitting, by splitting training data into training and validation data. Overfitting occurs when the model is learning the training data too well, causing the training loss to continuously decrease, while the validation loss continuously increases.

The mechanism for splitting the data is rather naive, as it takes the first x images as training data, and the remaining 1-x as validation data. A future enhancement to this would be to randomise the selection process, as this could boost model accuracy, due to structural difference in images towards the end of the dataset.

This allows for each epoch to run a 'training epoch' and a 'validation epoch', as shown in the code snippet below.

As epochs progress, we monitor the training and validation loss, storing values into an array. Doing this allows us to plot a graph of the training/validation loss over time, as shown below.

In [ ]:
# Run Epochs for training
# StepLR sets the learning rate of each parameter group to the initial lr decayed by gamma every step_size epochs.
scheduler = StepLR(optimizer, step_size, lr_gamma) 
current_epoch = 0
# record loss overtime for plotting
train_loss_ot = []
val_loss_ot = []

print("Training on " + str(len(train_data)))
print("Validating on " + str(len(val_data)))
    
# run model epochs
for epoch in range(current_epoch, epochs):
    scheduler.step(epoch)
    print("Epoch: " + str(epoch+1) + "/" + str(epochs))
    train_loss, train_time = training_epoch(epoch, model, train_loader, optimizer) # run a training epoch
    val_loss, val_time = validation_epoch(epoch, model, val_loader, optimizer) # run a validation epoch
    train_loss_ot.append(train_loss)
    val_loss_ot.append(val_loss)
    print(" Train Loss: " + str(train_loss) + " | Validation Loss: " + str(val_loss)) # print loss for the completeted epoch
    print("Train Time: " + str(train_time) + " | Validation Time: " + str(val_time))

#### insert image of some graphs here


### Performance Measure



To measure performance, we used training and validation loss, as discussed above, as well as SSIM.

SSIM (Structural Similarity Index Measure), is a measurement of how structurally similar two images are, making it a form of accuracy measure. This allows for a calculation to be made regarding the average 'accuracy' of the model's generated images, and their corresponding ground truths.

In [ ]:
def ssim(gt, pred):
    """ Compute Structural Similarity Index Metric (SSIM). Required 3D input np arrays"""
    return compare_ssim(
        gt.transpose(1,2,0), pred.transpose(1,2,0), multichannel=True, data_range=gt.max()
    )  

# calculate average ssim for training and validation data
length = len(gts)
i = 0
ssim_comb = 0
for i in range(0,length):
    ssim_comb += ssim(gts[i], preds[i])

ssim = ssim_comb / length

print("Average SSIM: " + str(ssim))

# Experiments
(1350 - 1800 words)

Note: This is just an example of the kind of layout and stuff to put in this section...we'll need to choose the most worthwhile experimentations to talk about

### Experimenting With Hyperparmeters

| Epochs  | Learning Rate | Dropout Probability  | Step Size | Training Loss  | Validation Loss | SSIM |
|---|---|---|---|---|---|---|
|30 |0.01|0.001|15|0.0488 |0.0448|0.41407|
|20 |0.1|0.001|15|0.056 |0.054 |0.41165|
|20 |0.1|0.01 |15|0.0685|0.0642|0.42711|
|50 |0.1|0.01 |15|0.0659|0.0641|0.41018|
|75 |0.1|0.01 |15|0.0648|0.0622|0.42522|
|20 |0.1|0.01 |25|0.1144|0.1141|0.38437|
|75 |0.1|0.01 |25|0.0581|0.0549|0.46331|




As noticable from the table above, a low training/validation loss does not necessarily mean the average SSIM of the images will be higher...

#  Conclusions
(300 - 400 words)

# Description of Contribution
(150 - 200 words)

# Reference List

## References
1. https://arxiv.org/pdf/1811.08839.pdf
2. https://arxiv.org/ftp/arxiv/papers/1704/1704.06825.pdf